# Question 4: General Theory/Application

_No need to be verbose, it's not fun for anyone_

1. What part of S**L**A**M** did this project deal with? Why? What does the other part deal with and how would it generally work, given that you only have LIDAR scans, RGB video stream, and noisy pose data for a moving robot?


2. Loop closures play an important role in reducing drift, how would you go about detecting these?


3. Explain the structure of your Jacobian. Is the pose-graph fully connected? Why/Why not?


4. With what you know now, how would you describe and differentiate the SLAM frontend and backend? Why do we need to optimise our poses/map in the first place - where does the noise come from/why?

_Your Answer_

### Answer 1

This project deals with the **SLAM Backend**. We have the odometry and sensor data available to us, as well as the predicted trasforms between the sensed images using ICP or similar algorithm. This project starts with the SLAM problem after the SLAM frontend has generated an initial estimate of the pose transforms between the different observations, and has some belief state on the n points, and the SLAM backend serves to **optimize out the errors** using Loop closures as a trick.

Given many different kinds of sensors, like LIDAR, RGB, Odometry, etc. we can use each of them to compute the relative pose independently using an ICP like algorithm which will be a part of the SLAM frontend. Then we write all that data as constraints and attach appropriate weights to them, and feed it to the SLAM backend, where the ICP SLAM algorithm helps find coordinates which minimizes the weighted loss from all those sensors.

### Answer 2

There are many possible methods to detect loop closures, some examples are given below:
* Move along the edges of the environment, detect loop closure on clearly visible corners the surroundings of which are clearly mapped.
* Try to find image correspondences in the two position of exact landmark features like a set of vertices in the visible objects.

### Answer 3

The Jacobian is spase. The graph is a **single connected component**, but it's **not fully connected**, it is infact **very sparse**. This is so because we have only connected the consecutive points in the graph and the loop closures are connected, so in our jacobian only those terms which for each constraint involve the right variable are non-zero, the others are all zero.

So each row has only very few, i.e. 3-6 typically non-zero elements.

Our Jacobian is the derivative of all of our residuals, i.e. the terms we want to minimize, with respect to the  :

\begin{bmatrix}
    \frac{\partial \text{anchor $x$}}                   {\partial x_1} & \frac{\partial \text{anchor $x$}}                   {\partial y_1} & \frac{\partial \text{anchor $x$}}                   {\partial \theta_1} & \frac{\partial \text{anchor $x$}}                   {\partial x_2} & \ldots & \frac{\partial \text{anchor $x$}}                   {\partial theta_n} \\
    \frac{\partial \text{anchor $y$}}                   {\partial x_1} & \frac{\partial \text{anchor $y$}}                   {\partial y_1} & \frac{\partial \text{anchor $y$}}                   {\partial \theta_1} & \frac{\partial \text{anchor $y$}}                   {\partial x_2} & \ldots & \frac{\partial \text{anchor $y$}}                   {\partial theta_n} \\
    \frac{\partial \text{anchor $\theta$}}              {\partial x_1} & \frac{\partial \text{anchor $\theta$}}              {\partial y_1} & \frac{\partial \text{anchor $\theta$}}              {\partial \theta_1} & \frac{\partial \text{anchor $\theta$}}              {\partial x_2} & \ldots & \frac{\partial \text{anchor $\theta$}}              {\partial theta_n} \\
    \frac{\partial \text{odometry edge 1 $x$}}          {\partial x_1} & \frac{\partial \text{odometry edge 1 $x$}}          {\partial y_1} & \frac{\partial \text{odometry edge 1 $x$}}          {\partial \theta_1} & \frac{\partial \text{odometry edge 1 $x$}}          {\partial x_2} & \ldots & \frac{\partial \text{odometry edge 1 $x$}}          {\partial theta_n} \\
    \frac{\partial \text{odometry edge 1 $y$}}          {\partial x_1} & \frac{\partial \text{odometry edge 1 $y$}}          {\partial y_1} & \frac{\partial \text{odometry edge 1 $y$}}          {\partial \theta_1} & \frac{\partial \text{odometry edge 1 $y$}}          {\partial x_2} & \ldots & \frac{\partial \text{odometry edge 1 $y$}}          {\partial theta_n} \\
    \frac{\partial \text{odometry edge 1 $\theta$}}     {\partial x_1} & \frac{\partial \text{odometry edge 1 $\theta$}}     {\partial y_1} & \frac{\partial \text{odometry edge 1 $\theta$}}     {\partial \theta_1} & \frac{\partial \text{odometry edge 1 $\theta$}}     {\partial x_2} & \ldots & \frac{\partial \text{odometry edge 1 $\theta$}}     {\partial theta_n} \\
    \frac{\partial \text{odometry edge 2 $x$}}          {\partial x_1} & \frac{\partial \text{odometry edge 2 $x$}}          {\partial y_1} & \frac{\partial \text{odometry edge 2 $x$}}          {\partial \theta_1} & \frac{\partial \text{odometry edge 2 $x$}}          {\partial x_2} & \ldots & \frac{\partial \text{odometry edge 2 $x$}}          {\partial theta_n} \\
    \frac{\partial \text{odometry edge 2 $y$}}          {\partial x_1} & \frac{\partial \text{odometry edge 2 $y$}}          {\partial y_1} & \frac{\partial \text{odometry edge 2 $y$}}          {\partial \theta_1} & \frac{\partial \text{odometry edge 2 $y$}}          {\partial x_2} & \ldots & \frac{\partial \text{odometry edge 2 $y$}}          {\partial theta_n} \\
    \frac{\partial \text{odometry edge 2 $\theta$}}     {\partial x_1} & \frac{\partial \text{odometry edge 2 $\theta$}}     {\partial y_1} & \frac{\partial \text{odometry edge 2 $\theta$}}     {\partial \theta_1} & \frac{\partial \text{odometry edge 2 $\theta$}}     {\partial x_2} & \ldots & \frac{\partial \text{odometry edge 2 $\theta$}}     {\partial theta_n} \\
    \ldots\\
    \frac{\partial \text{odometry edge $n_e$ $x$}}      {\partial x_1} & \frac{\partial \text{odometry edge $n_e$ $x$}}      {\partial y_1} & \frac{\partial \text{odometry edge $n_e$ $x$}}      {\partial \theta_1} & \frac{\partial \text{odometry edge $n_e$ $x$}}      {\partial x_2} & \ldots & \frac{\partial \text{odometry edge $n_e$ $x$}}      {\partial theta_n} \\
    \frac{\partial \text{odometry edge $n_e$ $y$}}      {\partial x_1} & \frac{\partial \text{odometry edge $n_e$ $y$}}      {\partial y_1} & \frac{\partial \text{odometry edge $n_e$ $y$}}      {\partial \theta_1} & \frac{\partial \text{odometry edge $n_e$ $y$}}      {\partial x_2} & \ldots & \frac{\partial \text{odometry edge $n_e$ $y$}}      {\partial theta_n} \\
    \frac{\partial \text{odometry edge $n_e$ $\theta$}} {\partial x_1} & \frac{\partial \text{odometry edge $n_e$ $\theta$}} {\partial y_1} & \frac{\partial \text{odometry edge $n_e$ $\theta$}} {\partial \theta_1} & \frac{\partial \text{odometry edge $n_e$ $\theta$}} {\partial x_2} & \ldots & \frac{\partial \text{odometry edge $n_e$ $\theta$}} {\partial theta_n} \\
    \frac{\partial \text{loop closure 1 $x$}}           {\partial x_1} & \frac{\partial \text{loop closure 1 $x$}}           {\partial y_1} & \frac{\partial \text{loop closure 1 $x$}}           {\partial \theta_1} & \frac{\partial \text{loop closure 1 $x$}}           {\partial x_2} & \ldots & \frac{\partial \text{loop closure 1 $x$}}           {\partial theta_n} \\
    \frac{\partial \text{loop closure 1 $y$}}           {\partial x_1} & \frac{\partial \text{loop closure 1 $y$}}           {\partial y_1} & \frac{\partial \text{loop closure 1 $y$}}           {\partial \theta_1} & \frac{\partial \text{loop closure 1 $y$}}           {\partial x_2} & \ldots & \frac{\partial \text{loop closure 1 $y$}}           {\partial theta_n} \\
    \frac{\partial \text{loop closure 1 $\theta$}}      {\partial x_1} & \frac{\partial \text{loop closure 1 $\theta$}}      {\partial y_1} & \frac{\partial \text{loop closure 1 $\theta$}}      {\partial \theta_1} & \frac{\partial \text{loop closure 1 $\theta$}}      {\partial x_2} & \ldots & \frac{\partial \text{loop closure 1 $\theta$}}      {\partial theta_n} \\
    \frac{\partial \text{loop closure 2 $x$}}           {\partial x_1} & \frac{\partial \text{loop closure 2 $x$}}           {\partial y_1} & \frac{\partial \text{loop closure 2 $x$}}           {\partial \theta_1} & \frac{\partial \text{loop closure 2 $x$}}           {\partial x_2} & \ldots & \frac{\partial \text{loop closure 2 $x$}}           {\partial theta_n} \\
    \frac{\partial \text{loop closure 2 $y$}}           {\partial x_1} & \frac{\partial \text{loop closure 2 $y$}}           {\partial y_1} & \frac{\partial \text{loop closure 2 $y$}}           {\partial \theta_1} & \frac{\partial \text{loop closure 2 $y$}}           {\partial x_2} & \ldots & \frac{\partial \text{loop closure 2 $y$}}           {\partial theta_n} \\
    \frac{\partial \text{loop closure 2 $\theta$}}      {\partial x_1} & \frac{\partial \text{loop closure 2 $\theta$}}      {\partial y_1} & \frac{\partial \text{loop closure 2 $\theta$}}      {\partial \theta_1} & \frac{\partial \text{loop closure 2 $\theta$}}      {\partial x_2} & \ldots & \frac{\partial \text{loop closure 2 $\theta$}}      {\partial theta_n} \\
    \ldots\\
    \frac{\partial \text{loop closure $n_l$ $x$}}       {\partial x_1} & \frac{\partial \text{loop closure $n_l$ $x$}}       {\partial y_1} & \frac{\partial \text{loop closure $n_l$ $x$}}       {\partial \theta_1} & \frac{\partial \text{loop closure $n_l$ $x$}}       {\partial x_2} & \ldots & \frac{\partial \text{loop closure $n_l$ $x$}}       {\partial theta_n} \\
    \frac{\partial \text{loop closure $n_l$ $y$}}       {\partial x_1} & \frac{\partial \text{loop closure $n_l$ $y$}}       {\partial y_1} & \frac{\partial \text{loop closure $n_l$ $y$}}       {\partial \theta_1} & \frac{\partial \text{loop closure $n_l$ $y$}}       {\partial x_2} & \ldots & \frac{\partial \text{loop closure $n_l$ $y$}}       {\partial theta_n} \\
    \frac{\partial \text{loop closure $n_l$ $\theta$}}  {\partial x_1} & \frac{\partial \text{loop closure $n_l$ $\theta$}}  {\partial y_1} & \frac{\partial \text{loop closure $n_l$ $\theta$}}  {\partial \theta_1} & \frac{\partial \text{loop closure $n_l$ $\theta$}}  {\partial x_2} & \ldots & \frac{\partial \text{loop closure $n_l$ $\theta$}}  {\partial theta_n} \\
\end{bmatrix}


### Answer 4

![SLAM Frontend and Backend](../misc/slam-frontend-backend.png)

The SLAM algorithm has 2 parts:
* The SLAM Frontend includes the sensor information gatering from LIDAR, Odometry, RGBD Cameras and pose estimation by ICP algorithm, etc. It also involves detecting loop closures, etc.
* The SLAM Backend involves taking this pose and object data and generation the maps and the localization information from it.

We need to correct for noise since it comes from a variety of sources:
* The real world is moving, so our observations correspondences aren't guarenteed to be correct. One object moving slightly in a room makes the problem underdefined.
* The sensors have several sources of errors. Motors aren't perfect, wheels skid a little, cameras aren't infinitely stable, and all small errors accumulate over time to become significant.